In [73]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
import math

import time

In [74]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv')

In [75]:
# No SVD, o 0 representa valores desconhecidos
def svd_algebra(train):
    global_mean = train["rating"].mean()
    users = pd.Series(range(1,train["user_id"].max()+1), name="user_id")
    movies = pd.Series(range(1,train["movie_id"].max()+1), name="movie_id")
    df_u_m = train.pivot(index='user_id', columns='movie_id', values='rating')
    df_u_m = pd.DataFrame(df_u_m, index=users, columns=movies)
    # df_u_m.fillna(value=global_mean, inplace=True)
    df_u_m.fillna(value=0, inplace=True)
    U, s, Vh = np.linalg.svd(df_u_m.values)
    return { "u": U, "s": s, "v": Vh }

In [76]:
def predict(model, u,i, k = 5):
    p2 = model["u"][u, :k]
    q2 = model["v"][i, :k]
    s2 = model["s"][:k]
    return p2.dot(np.diag(s2)).dot(q2)

In [77]:
# Avaliando predições
def rmse(model, test, k = 5):
    sum_err = 0
    for t in test:
        u = t[0]-1
        i = t[1]-1
        r_ui = t[2]
        pred = predict(model,u, i, k)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [78]:
def validation(model, data, train_split = 0.75, k = 5):
    data = data.sample(frac=1)
    train_len = int(train_split * len(data))
    train = data[:train_len]
    test = data[train_len:].values

    start_time = time.time()
    model = model(train)
    print("Tempo de treinamento em segundos: ", time.time() - start_time)
    start_time = time.time()
    print("RMSE :", rmse(model, test, k))
    print("Tempo de predição em segundos: ", time.time() - start_time)

In [79]:
# rmse(df.values[:10])

In [80]:
validation(svd_algebra, train, 0.75,k = 5)

Tempo de treinamento em segundos:  30.423816680908203
RMSE : 3.837957981302117
Tempo de predição em segundos:  1.5704355239868164


## Execução 1 treino 75% k = 10 (misturando todos os usuários) e média global
Tempo de treinamento em segundos:  25.135908365249634 <br />
RMSE : 5.3744788621470185 <br />
Tempo de predição em segundos:  1.3862569332122803 <br />

## Execução 2 treino 75% k = 10 (misturando todos os usuários) e com 0
Tempo de treinamento em segundos:  27.009052991867065 <br />
RMSE : 3.8223417939227353 <br />
Tempo de predição em segundos:  1.4589087963104248 <br />

## Execução 3 treino 75% k = 100 (misturando todos os usuários) e com 0
Tempo de treinamento em segundos:  31.526059865951538 <br />
RMSE : 3.8395565904734363 <br />
Tempo de predição em segundos:  9.630928993225098 <br />

## Execução 4 treino 75% k = 1000 (misturando todos os usuários) e com 0
Tempo de treinamento em segundos:  31.530203580856323 <br />
RMSE : 3.8822722385473316 <br />
Tempo de predição em segundos:  407.36043882369995 <br />

## Execução 5 treino 75% k = 5 (misturando todos os usuários) e com 0
Tempo de treinamento em segundos:  30.864771842956543 <br />
RMSE : 3.8119544174363806 <br /> 
Tempo de predição em segundos:  1.5278677940368652 <br />

## Execução 6 treino 75% k = 5 (misturando todos os usuários) e com média do item
Tempo de treinamento em segundos:  29.900591373443604 <br />
RMSE : 5.0837250874946465 <br />
Tempo de predição em segundos:  1.515906572341919 <br />